In [2]:
import tensorflow as tf
import tensorboard
import matplotlib.pyplot as plt
self.writer = tf.summary.create_file_writer(loader_log_dir)

def log_vocabulary(self):
        with self.writer.as_default():
            for word, index in self.problem_tokenizer.word_index.items():
                tf.summary.text(name="Problem Vocabulary", data=f"{word}: {index}", step=0)

            for word, index in self.solution_tokenizer.word_index.items():
                tf.summary.text(name="Solution Vocabulary", data=f"{word}: {index}", step=0)

            self.writer.flush()

2024-02-08 03:11:37.071144: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-08 03:11:37.113323: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 03:11:37.113343: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 03:11:37.113417: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-08 03:11:37.123413: I tensorflow/core/platform/cpu_feature_g

NameError: name 'loader_log_dir' is not defined

In [ ]:
def _log_dataset_samples(self, problems, decoder_inputs, targets):
        with self.writer.as_default():
            for i, (problem, decoder_input, target) in enumerate(zip(problems, decoder_inputs, targets)):
                if i >= 5: # Log 5 samples
                    break
                
                # Convert padded sequences back to text
                problem_text = self.problem_tokenizer.sequences_to_texts([problem])
                decoder_input_text = self.solution_tokenizer.sequences_to_texts([decoder_input])
                target_text = self.solution_tokenizer.sequences_to_texts([target])

                # Truncate texts
                #max_display_length = 1024
                #problem_text = (problem_text[:max_display_length] + '...') if len(problem_text) > max_display_length else problem_text
                #solution_text = (solution_text[:max_display_length] + '...') if len(solution_text) > max_display_length else solution_text

                # Log to TensorBoard
                tf.summary.text(name=f"problem_{i}", data=problem_text, step=0)
                tf.summary.text(name=f"decoder_input_{i}", data=decoder_input_text, step=0)
                tf.summary.text(name=f"target_{i}", data=target_text, step=0)

            self.writer.flush()

In [3]:
class Evaluator:
    def __init__(self, model, loader):
        self.model = model
        self.loader = loader

    def plot_loss(self, history):
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['loss'])
        plt.title('Loss Curve')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.show()

    def plot_token_probabilities(self, token_index, n_samples=1):
        # Take one batch from the dataset
        for (encoder_input, decoder_input), _ in self.loader.dataset.take(1):
            # Slice the batch down to n_samples
            encoder_input = encoder_input[:n_samples]
            decoder_input = decoder_input[:n_samples]

            # Predict on the sliced inputs
            predictions = self.model.predict([encoder_input, decoder_input])

            for sample_idx in range(n_samples):
                # For each sample, extract token logits and convert to probabilities
                token_logits = predictions[sample_idx, token_index, :]
                token_probabilities = tf.nn.softmax(token_logits).numpy()
                sorted_indices = np.argsort(token_probabilities)[::-1]
                sorted_probabilities = token_probabilities[sorted_indices]

                # Plotting
                plt.figure(figsize=(20, 5))
                plt.bar(range(len(sorted_probabilities)), sorted_probabilities)
                plt.xlabel('Word Indices (sorted by probability)')
                plt.ylabel('Probability')
                plt.title(f'Word Prediction Probabilities for Token {token_index} in Sample {sample_idx+1}')
                plt.show()

    def generate_training_predictions(self, n_samples=1):
        # Take one batch from the dataset
        for (encoder_inputs, decoder_inputs), _ in self.loader.dataset.take(1):
            # Slice the batch down to n_samples
            encoder_inputs = encoder_inputs[:n_samples]
            decoder_inputs = decoder_inputs[:n_samples]

            # Predict on the sliced inputs
            predictions = self.model.predict([encoder_inputs, decoder_inputs])
            predicted_sequences = np.argmax(predictions, axis=-1)

            # Convert sequences to text
            encoder_inputs = encoder_inputs = encoder_inputs.numpy() # Can factor this later
            input_texts = self.loader.problem_tokenizer.sequences_to_texts(encoder_inputs)
            predicted_texts = self.loader.solution_tokenizer.sequences_to_texts(predicted_sequences)
            
            # Print each prediction in the slice
            for i, predicted_text in enumerate(predicted_texts):
                print(f"Sample {i + 1}:") 
                print("Input sequence [:250]:", input_texts[i][:250])
                print("Predicted sequence:", predicted_sequences[i])
                print("Predicted text:", predicted_text, "\n")
    
    def generate_manual_predictions(self, input_text):
        # Tokenize the input string
        input_seq = self.loader.problem_tokenizer.texts_to_sequences([input_text])
        input_padded = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=self.loader.max_length_input, padding='post')

        # Prepare the decoder input
        start_token_index = self.loader.solution_tokenizer.word_index.get('[START]', 1)  # Fallback to 1 if not found
        decoder_input = np.array([[start_token_index]])
        
        # Generate and interpret the prediction
        predictions = self.model.predict([input_padded, decoder_input])
        predicted_sequence = np.argmax(predictions, axis=-1)[0]
        predicted_text = self.loader.solution_tokenizer.sequences_to_texts([predicted_sequence])
        
        # Print the output
        print("Input text:", input_text)
        print("Predicted text:", predicted_text[0])

    def evaluate(self, command, *args, **kwargs):
        if command == 'loss':
            self.plot_loss(*args, **kwargs)
        elif command == 'token_prob':
            self.plot_token_probabilities(*args, **kwargs)
        elif command == 'training_sample_pred':
            self.generate_training_predictions(*args, **kwargs)
        elif command == 'manual_sample_pred':
            self.generate_manual_predictions(*args, **kwargs)
        else:
            print(f"Unknown command: {command}")

# Load the model if it's not
model = tf.keras.models.load_model('/workspace')

# Uncomment what you want to run
evaluator = Evaluator(model, loader)
#evaluator.evaluate('loss', history)
#evaluator.evaluate('token_prob', token_index=10, n_samples=3)
evaluator.evaluate('training_sample_pred', n_samples=3)

input_text = "XXSTATEMENT Susie like squares.  When playing with one, she wants to know how many sides it has.  For each side of the square, print its length.  XXINPUT An object s, which is an integer representing the length of a side l.  XXOUTPUT Four copies of the length s"
evaluator.evaluate('manual_sample_pred', input_text)


2024-02-08 03:11:45.434094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-08 03:11:45.453784: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-08 03:11:45.453830: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-08 03:11:45.462164: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-08 03:11:45.462197: I tensorflow/compile

NameError: name 'loader' is not defined

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs --host localhost --port 8088